In [1]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import matplotlib.pyplot as plt
import keras_cv
from keras_cv.models import StableDiffusion
from keras_cv.models.stable_diffusion.diffusion_model import DiffusionModel
import tensorflow.keras as keras
import time


SAVE_PATH = "/users/skoka/Documents/Final_Paper_ML"


Stable_diffusion = StableDiffusion(img_height=512, img_width=512)
decoder = Stable_diffusion.decoder

diffusion_model = DiffusionModel(img_width = 512, img_height = 512, max_text_length=0)

import math
def get_timestep_embedding(timestep, batch_size, dim=320, max_period=10000):
        half = dim // 2
        freqs = tf.math.exp(
            -math.log(max_period) * tf.range(0, half, dtype=tf.float32) / half
        )
        args = tf.convert_to_tensor([timestep], dtype=tf.float32) * freqs
        embedding = tf.concat([tf.math.cos(args), tf.math.sin(args)], 0)
        embedding = tf.reshape(embedding, [1, -1])
        return tf.repeat(embedding, batch_size, axis=0)

# we are going to need to modify this code a lot 
def generate_pattern(layer_name, filter_index, size=64):
    # Build a model that outputs the activation
    # of the nth filter of the layer considered.
    layer_output = diffusion_model.get_layer(layer_name).output
    # Isolate the output 
    new_model = tf.keras.models.Model(inputs=diffusion_model.inputs, outputs=layer_output)
    
    # We start from a gray image with some uniform noise
    input_img_data = np.random.random((1, size, size, 4)) * 20 + 128.
    I = tf.Variable(input_img_data, name='image_var', dtype = 'float64')
    #I = preprocess_input(I_start) # only process once
    # Run gradient ascent for 40 steps
    eta = 5
    for i in range(100):
        start = time.time()
        with tf.GradientTape(watch_accessed_variables=False) as tape:
            time_embedding = get_timestep_embedding(i, 1)
            # create a tensor with the following size of all zeros: (None, 0, 768)
            word_embedding = tf.zeros((1, 0, 768))
            tape.watch(I)
            # get variable to maximize
            model_vals = new_model((I, time_embedding, word_embedding))
            loss = tf.reduce_mean(model_vals[:, :, :, filter_index])

        # Compute the gradient of the input picture w.r.t. this loss
        # add this operation input to maximize
        grad_fn = tape.gradient(loss, I)
        # Normalization trick: we normalize the gradient
        grad_fn /= (tf.sqrt(tf.reduce_mean(tf.square(grad_fn))) + 1e-5) # mean L2 norm
        I = I + (grad_fn * eta) # one iteration of maximizing
        end = time.time()
        print("Iteration: {}, Loss: {}, Time: {}".format(i, loss, end-start))
    # decode the resulting input image
    I = decoder(I)
    # return the numpy matrix so we can visualize 
    img = I.numpy()
    return img

2023-04-27 22:56:39.692751: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 22:56:42.013685: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/cudnn-8.2.4.15-11.4-eluwegpwn6adr7hlku5p5wru5xzefpop/lib64:/hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/cuda-11.7.0-vbhdtgc7dl4kpo4auyswsh6w3udcnf5x/lib64:/hpc/mp/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-10.3.0/neovim-0.7.0-terkir3wk5rst6ktv4uxyaqjditacv5p/lib
2023-04-27 22:56:42.013834: W tensorflow/compiler/xla/stream_execut

2.11.0
You do not have pycocotools installed, so KerasCV pycoco metrics are not available. Please run `pip install pycocotools`.
You do not have pyococotools installed, so the `PyCOCOCallback` API is not available.
You do not have Waymo Open Dataset installed, so KerasCV Waymo metrics are not available.


/users/skoka/.venv/MLEnv3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [46]:
cnn_layers_in_order = []

for layer in range(len(diffusion_model.layers)):
    curr_layer = diffusion_model.layers[layer]
    if len(curr_layer.get_weights()) != 0:
        for sublayer in range(len(curr_layer.get_weights())):
            shape = curr_layer.get_weights()[sublayer].shape
            if len(shape) == 4 and shape[0] == 3 and shape[1] == 3:
                cnn_layers_in_order.append((layer, sublayer))

In [49]:
cnn_layers_in_order

[(4, 0),
 (6, 2),
 (6, 8),
 (9, 2),
 (9, 8),
 (11, 0),
 (12, 2),
 (12, 8),
 (14, 2),
 (14, 8),
 (16, 0),
 (17, 2),
 (17, 8),
 (19, 2),
 (19, 8),
 (21, 0),
 (22, 2),
 (22, 8),
 (23, 2),
 (23, 8),
 (24, 2),
 (24, 8),
 (26, 2),
 (26, 8),
 (28, 2),
 (28, 8),
 (30, 2),
 (30, 8),
 (32, 2),
 (32, 8),
 (33, 0),
 (35, 2),
 (35, 8),
 (38, 2),
 (38, 8),
 (41, 2),
 (41, 8),
 (43, 0),
 (45, 2),
 (45, 8),
 (48, 2),
 (48, 8),
 (51, 2),
 (51, 8),
 (53, 0),
 (55, 2),
 (55, 8),
 (58, 2),
 (58, 8),
 (61, 2),
 (61, 8),
 (65, 0)]

In [65]:
def l2_norm(layer_weights):
    num_filters = layer_weights.shape[3]
    norms_list = []
    prev_filters_list = []
    curr_filters_list = []
    for filter in range(num_filters):
        l2_norms = np.linalg.norm(layer_weights[:,:,:,filter], axis = (0,1))
        # get max l2 norm
        norms_list.append(np.max(l2_norms))
        curr_filters_list.append(filter)
        prev_filters_list.append(np.argmax(l2_norms))
    norms_list = np.array(norms_list)
    prev_filters_list = np.array(prev_filters_list)
    index = np.argmax(norms_list)
    return norms_list.max(), prev_filters_list[index], curr_filters_list[index]
    

In [66]:
for conv in cnn_layers_in_order:
    layer = conv[0]
    sublayer = conv[1]
    layer_weights = diffusion_model.layers[layer].get_weights()[sublayer]
    print(l2_norm(layer_weights))

(0.7164252, 2, 235)
(0.76154906, 73, 68)
(0.5539933, 202, 124)
(0.7220448, 45, 163)
(0.55759263, 37, 11)
(0.5239288, 235, 11)
(3.4893906, 316, 234)
(0.49602467, 184, 277)
(1.1618814, 521, 301)
(1.0276113, 301, 521)
(0.5614104, 632, 343)
(6.849683, 174, 36)
(0.936096, 636, 84)
(1.5780002, 302, 17)
(0.7110146, 543, 959)
(0.7115272, 1265, 225)
(1.3082315, 283, 747)
(1.1329187, 191, 713)
(1.4866389, 1069, 587)
(1.5052186, 1034, 195)
(1.0229278, 195, 772)
(0.9280076, 854, 1002)
(1.3937745, 998, 1197)
(1.6156629, 1058, 573)
(1.2737345, 1684, 511)
(1.5857177, 854, 270)
(2.7754679, 734, 960)
(1.8347987, 960, 468)
(2.010588, 623, 37)
(0.75336003, 510, 172)
(0.48448202, 741, 579)
(2.9103, 579, 785)
(3.8729475, 1021, 477)
(3.6221933, 901, 507)
(1.8498381, 133, 1222)
(10.560751, 310, 687)
(0.82729536, 1250, 1220)
(1.191778, 58, 361)
(3.0310528, 802, 128)
(1.4015703, 202, 250)
(2.0739264, 478, 524)
(1.3216797, 484, 266)
(1.4742858, 526, 184)
(0.5908569, 454, 309)
(0.49592996, 447, 559)
(1.8925539, 

In [88]:
filter = diffusion_model.layers[cnn_layers_in_order[51][0]].get_weights()[cnn_layers_in_order[51][1]][:,:,303,3]

In [96]:
def get_circuit(layer, sublayer, prev_layer_filter, curr_layer_channel):
    curr_layer_index = len(cnn_layers_in_order) - 1
    prev_layer_index = curr_layer_index - 1
    prev_layer_multi_channel_filter = diffusion_model.layers[cnn_layers_in_order[prev_layer_index][0]].get_weights()[cnn_layers_in_order[prev_layer_index][1]][:,:,prev_layer_filter,:]
    


get_circuit(cnn_layers_in_order[51][0], cnn_layers_in_order[51][1], 303, 3)

(3, 3, 320)


In [94]:
filter.shape

(3, 3)